# Full workflow

This notebook demonstrates the full workflow. The steps include:
1. Create a virtual library, we will do this exemplary for monosubstituted thiophenes
2. Download the models from Zenodo
3. Predict the triplet energy and the spin density using the models
4. Use an automated analysis to find the most promising candidates
5. Visualize the results using an interactive analysis

In order to run this notebook, you need to have cloned this repository and installed the required packages

# Create the virtual space
First we want to create the virtual library. For this you need to give the name of the directory where the project is located at.

In [ ]:
import sys
import pandas as pd

home_dir = r'\YOUR_PATH\EnT_Substrate_Mapping\\' #Change this to the directory your project is located at.
sys.path.append(home_dir) #Add the directory to your pythonpath, so that the functions from Combinatorial.help_to_combine can be imported.
from CombinatorialSpace.help_to_combine import get_functional_groups, create_space

path_to_fg_list = home_dir + r'/CombinatorialSpace/functional_groups.txt' #This file contains the functional groups that can be added to the core. You can add your own functional groups to this file.
functional_groups = get_functional_groups(path_to_fg_list)

Here we generate a virtual library of monosubstituted thiophenes. For other heterocycles the corresponding smiles needs to be given. The number for num_iterations determines how many times the functional groups are added to the core. The keep_all parameter determines whether all possible combinations of functional groups are kept or only the first one. To get the space of disubstituted molecules set num_iterations=2 and for the triubstituted to num_iterations=3.

In [ ]:
smiles = 'C1=CC=CS1'
ring_class = 'thiophene'
substitution = 'monosubstituted'

virtual_space = create_space(functional_groups, smiles, smiles, num_iterations=1, keep_all=True)
df = pd.DataFrame(data={'smiles': virtual_space})
df.to_csv(home_dir + f'/Data/virtual_libraries/{ring_class}_{substitution}.csv', index=False, columns=['smiles'])

Now that we've created the chemical space we want to predict the triplet energy and the spin density. For this we first download the models from Zenodo. Depending on your internet connection this might take a few minutes.

In [ ]:
import requests
zenodo_file_url = 'https://zenodo.org/records/10391170/files/spin_population.pt?download=1'
output_filename = home_dir + 'Models/spin_population/spin_population.pt'
response = requests.get(zenodo_file_url, stream=True)

# Check if the download was successful
if response.status_code == 200:
    with open(output_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded successfully as {output_filename}")
else:
    print(f"Failed to download. Status code: {response.status_code}")

zenodo_file_url = 'https://zenodo.org/records/10391170/files/triplet_energy.pt?download=1'
output_filename = home_dir + 'Models/triplet_energy/triplet_energy.pt'
response = requests.get(zenodo_file_url, stream=True)

# Check if the download was successful
if response.status_code == 200:
    with open(output_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded successfully as {output_filename}")
else:
    print(f"Failed to download. Status code: {response.status_code}")

Now that we have our virtual space and the models we can use the models for predictions. Let's start with the triplet energy:

In [ ]:
import contextlib
import chemprop

path_to_model = home_dir + 'Models/triplet_energy/triplet_energy.pt'
path_to_molecules = home_dir + f'Data/virtual_libraries/{ring_class}_{substitution}.csv'
path_for_predictions = home_dir + f'Data/et_predictions/predictions_{ring_class}_{substitution}.csv'

arguments = [
    '--test_path', path_to_molecules,
    '--preds_path', path_for_predictions,
    '--num_workers', '1',
    '--checkpoint_path', path_to_model,
    '--features_generator', 'rdkit_2d_normalized',
    '--no_features_scaling'
]

with contextlib.redirect_stdout(None):
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)

Let's continue with the spin population.

In [ ]:
import time
import torch
import pandas as pd

from EasyChemML.DataImport.DataImporter import DataImporter
from EasyChemML.DataImport.Module.CSV import CSV
from EasyChemML.Encoder import BertTokenizer
from EasyChemML.Environment import Environment
from EasyChemML.Encoder.impl_Tokenizer.SmilesTokenizer_SchwallerEtAll import SmilesTokenzier
from EasyChemML.Model.impl_Pytorch.Models.BERT.FP2MOL_BERT_Trans import FP2MOL_Bert

from rdkit import Chem

file_path = home_dir + f'Data/virtual_libraries/{ring_class}_{substitution}.csv'
path_to_model = home_dir + 'Models/spin_population/spin_population.pt'
path_for_predictions = home_dir + f'Data/sp_predictions/predictions_{ring_class}_{substitution}.csv'

# ----------------------------------- Data Preprocessing -----------------------
device = "cpu" # Change this to "cpu" if you don't have a GPU available.

###Do not change the following parameters unless you train a new model with different parameters
d_model = 512
heads = 4
N = 8
src_vocab_size =106
trg_vocab_size = 136
dropout = 0.1
max_seq_len = 100
###

# Load the model
model_object = FP2MOL_Bert(src_vocab_size, trg_vocab_size, N, heads, d_model, dropout, max_seq_len, device)
model_object.load_model_parameters(path_to_model)

env = Environment(WORKING_path_addRelativ='Output')

dataloader = {'data': CSV(file_path, columns=['smiles'])}
di = DataImporter(env)
bp = di.load_data_InNewBatchPartition(dataloader)

val_data_size = len(bp['data'])

print('Start BertTokenizer')
tokenizer = BertTokenizer()
tokenizer.convert(datatable=bp['data'], columns=['smiles'], n_jobs=4)

# ----------------------------------- Evaluation -----------------------

batch_size = 5
decoding_strategy = 'greedy'

s = SmilesTokenzier()
final_df=pd.DataFrame(data=None)

start_mol = 0
iter_steps = 0

start = time.time()
temp = start

iteration_per_chunk = int(val_data_size / batch_size)

for verbose, i in enumerate(range(int(iteration_per_chunk)+1)):

    end_mol = start_mol + batch_size

    true_smiSD = bp['data'][start_mol:end_mol]['smiles_ids']
    input_smi = bp['data'][start_mol:end_mol]['smiles_ids']
    input_smi = input_smi[:, 0:100]

    true_smi = bp['data'][start_mol:end_mol]['smiles']

    true_smiSD = torch.LongTensor(true_smiSD).to(torch.device(device))
    input_smi = torch.LongTensor(input_smi).to(torch.device(device))
    model_eval = model_object.fit_eval(input_smi, true_smiSD, method=decoding_strategy)
    outputs = model_eval.outputs
    start_mol = end_mol
    iter_steps += 1
    finished = False

    if decoding_strategy == 'beam_search':
        num_beams = 5
    else:
        num_beams = 1

    for example in range(batch_size):
        for beam in range(num_beams):
            counter = (example * num_beams) + beam
            pred_smi, pred_SD_string, DensityArray = s.getSmilesfromoutputwithSD(outputs[counter])

            df_example = pd.DataFrame({})
            df_example['Predicted_smiles'] = [pred_smi]
            df_example['Predicted_SD_string'] = [pred_SD_string]
            df_example['DensityArray'] = [DensityArray]
            df_example['True_smiles'] = [true_smi[example].decode("utf-8")]

            if pred_smi == str(true_smi[example].decode("utf-8")):
                df_example['Reconstrunction_error'] = [0]
            else:
                df_example['Reconstrunction_error'] = [1]

            final_df = pd.concat([final_df, df_example])
            if ((i*batch_size) + example+1) == val_data_size:
                finished = True
                break

        if finished:
            break

    print('----------------------------------')
    print(f'Done with {verbose+1} / {iteration_per_chunk} chunks')

    final_df.to_csv(path_for_predictions, index=False)

Now we generate the images for the spin density maps. For this we use the RDKit library. The images are saved in the folder Data/images.

In [ ]:
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import Draw

type_of_image = 'spin_population' # Options are structure or 'spin_population'
                            # Caveat of spin_population is that the memory for the generation for the images can overflow and crash the script.
                            # In this case the script needs to be run in smaller batches.

data_path = home_dir + f'/Data/virtual_libraries/{ring_class}_{substitution}.csv'
picFolder_path = home_dir + f'/Data/images/{ring_class}_{substitution}'
if not os.path.exists(picFolder_path):
    os.makedirs(picFolder_path)
#Only required for spin_density images:
sp_path = home_dir + f'/Data/sp_predictions/predictions_{ring_class}_{substitution}.csv'

df = pd.read_csv(data_path)

if not os.path.exists(picFolder_path):
    os.mkdir(picFolder_path)

if type_of_image == 'structure':
    for smi in df.smiles.to_list():
        d = Draw.rdMolDraw2D.MolDraw2DCairo(512, 512)
        d.drawOptions().rotate = 0
        d.drawOptions().bondLineWidth = 1
        d.DrawMolecule(Chem.MolFromSmiles(smi))
        d.FinishDrawing()
        idx = df[df['smiles'] == smi].index.to_numpy()[0]
        d.WriteDrawingText(f"{picFolder_path}/{idx}.jpeg")

elif type_of_image == 'spin_population':
    sp_data = pd.read_csv(sp_path)
    for i in range(len(sp_data)):
        smi = sp_data.True_smiles.to_list()[i]
        sp_list = sp_data.DensityArray.to_list()[i]
        string_list = sp_list.strip('[]')
        string_elements = string_list.split()
        sp_list = [int(element) for element in string_elements]
        m = Chem.MolFromSmiles(smi)
        densities = np.array(sp_list)
        count = df[df['smiles'] == smi].index.to_numpy()[0]
        fig = SimilarityMaps.GetSimilarityMapFromWeights(m,
                                                         densities,
                                                         colorMap='coolwarm',
                                                         contourLines=10,
                                                         colors='grey')
        fig.savefig(f"{picFolder_path}/{ring_class}_{substitution}/{count}.jpeg",
                    bbox_inches='tight',
                    dpi=600)
        fig.clear()

# Automated analysis

Now that we have predicted the triplet energy and the spin density we can use an automated analysis to find the most promising candidates. For this we will use define the following criteria:
1. The triplet energy is below a certain threshold (e.g. 62 kcal/mol)
2. The spin density is located on the core of the molecule (e.g. thiophene)

In [ ]:
threshold = 62 # Here we can define our triplet energy threshold

path_to_et_datafile = home_dir + fr'Data/et_predictions/predictions_{ring_class}_{substitution}.csv'
path_to_sp_datafile = home_dir + fr'Data/sp_predictions/predictions_{ring_class}_{substitution}.csv'
path_to_save_candidates = home_dir + f'Data/potential_candidates_{ring_class}_{substitution}.csv'

#--------------Load the data, and check if the maximum spin density is located on the core-------------------#
data_et = pd.read_csv(path_to_et_datafile)
data_sp = pd.read_csv(path_to_sp_datafile)

data_sp = data_sp.rename(columns={'True_smiles': 'smiles'})
data = data_sp.merge(data_et[['smiles', 'e_t']], on='smiles', how='left')

data['DensityArray'] = data['DensityArray'].apply(lambda x: eval(x.replace(' ', ',')))

density_located_on_core = []

for index, row in data.iterrows():
    predicted_smiles = row['Predicted_smiles']
    density_array = data[data['smiles'] == predicted_smiles]['DensityArray'].values[0]

    pattern_to_match = Chem.MolFromSmiles(smiles)
    atom_index_of_core = list(Chem.MolFromSmiles(predicted_smiles).GetSubstructMatch(pattern_to_match))

    density_on_core_list = []
    density_on_substituent_list = []
    for i in range(len(density_array)):
        if i in atom_index_of_core:
            density_on_core_list.append(density_array[i])
        else:
            density_on_substituent_list.append(density_array[i])

    if max(density_on_core_list) >= max(density_on_substituent_list):
        density_on_core=1
    else:
        density_on_core=0

    density_located_on_core.append(density_on_core)

data['Density_on_core'] = density_located_on_core


#--------------Select the molecules that have a predicted triplet energy below the threshold and where the spin density is located on the ring-------------#

interesting_data = data[data['e_t'] < threshold]
interesting_data = interesting_data[interesting_data['Density_on_core'] == 1]
interesting_data.to_csv(path_to_save_candidates)

We can have a look at the molecules that passed our criteria:

In [ ]:
interesting_data

For a more convenient analysis we can interactively analyse the results.

In [ ]:
import base64
import os, dash, glob
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from umap import UMAP
from dash import dash_table
import plotly.express as px
from dash.dependencies import Input, Output
from dash import html
from dash import dcc

#--------------Define which data shall be visualised and set the corresponding paths-------------------#

representation = 'ECFP' # Available options are ECFP, MACCS, MACCS+ECFP

picFolder_path = home_dir + f'Data/images/{ring_class}_{substitution}'
data_path = home_dir + f'Data/virtual_libraries/{ring_class}_{substitution}.csv'
e_t_path = home_dir + f'Data/et_predictions/predictions_{ring_class}_{substitution}.csv'

#--------------Load the data, compute representation and reduce dimensionality-------------------#

panda_dataframe = pd.read_csv(data_path)
panda_dataframe = pd.concat([panda_dataframe, pd.read_csv(e_t_path, usecols =['e_t'])], axis=1)
panda_dataframe['bins'] = pd.cut(panda_dataframe['e_t'], bins=np.arange(35,90,5), labels=np.arange(35,85,5))
panda_dataframe['IDS'] = panda_dataframe.index.to_list()

mol_list = [Chem.MolFromSmiles(smiles) for smiles in panda_dataframe['smiles']]

if representation == 'ECFP':
    fp_list = [AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024) for molecule in mol_list]
if representation == 'MACCS':
    fp_list = [MACCSkeys.GenMACCSKeys(molecule) for molecule in mol_list]
if representation == 'MACCS+ECFP':
    fp_list = [np.concatenate((MACCSkeys.GenMACCSKeys(molecule), AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024))) for molecule in mol_list]

FP_array = np.array(fp_list)
UMAPspace = UMAP(n_components=3,
                 n_neighbors=60,
                 min_dist=0.15,
                 metric='euclidean',
                 random_state=0,
                 n_jobs=1
                 ).fit(FP_array)

panda_dataframe['UMAP_1'] = UMAPspace.embedding_[:, 0]
panda_dataframe['UMAP_2'] = UMAPspace.embedding_[:, 1]
panda_dataframe['UMAP_3'] = UMAPspace.embedding_[:, 2]
data_array = panda_dataframe.to_numpy()

#--------------Load the images for easy analysis of the data. Differentiate three data situations-------------------#

globstatment = picFolder_path + '/*.jpeg'
imageFiles = glob.glob(globstatment)

if not imageFiles:                                                                                              # If no images are found in the folder, a dummy image is used
    print(f"No image files found in the folder: {picFolder_path} \n Using dummy image instead.")
    default_image = home_dir + r'\Data\images\default.png'
    pics = {}
    for id in panda_dataframe['IDS'].to_list():
        encoded_image = base64.b64encode(open(default_image, 'rb').read()).decode("utf-8")
        pics[id] = encoded_image

elif len(imageFiles) != len(data_array):                                                                        # If an inconsistend number of images is found (e.g, when the spin density map
    pics = {}                                                                                                   # is not available for all molecules), a dummy image is used in these cases
    print(f"Number of images in folder {picFolder_path} ({len(imageFiles)} images) does not match the number "
          f"of molecules in the dataset: {len(data_array)} \nUsing dummy image for missing images instead.")
    for id in panda_dataframe['IDS'].to_list():
        try:
            img_ = picFolder_path + f'/{id}.jpeg'
            encoded_image = base64.b64encode(open(img_, 'rb').read()).decode("utf-8")
        except:
            default_image = home_dir + r'\Data\images\default.png'
            encoded_image = base64.b64encode(open(default_image, 'rb').read()).decode("utf-8")
        pics[id] = encoded_image

else:                                                                                                            # If the number of images matches the number of molecules, the images are loaded normally
    pics = {}
    for i, file in enumerate(imageFiles):
        encoded_image = base64.b64encode(open(file, 'rb').read()).decode("utf-8")
        file_name = os.path.basename(file.split('.')[0])
        id = int(file_name)
        pics[id] = encoded_image

def generate_info_dataframe(mol_id):
    '''
    Function required for the interactive analysis. It generates a pandas dataframe containing
    the information about a molecule that is displayed in the table.
    '''
    index = int(np.where(data_transpose[:, 3] == mol_id)[0])
    mol_dataPoint = data_transpose[index]
    mol_SMILES = mol_dataPoint[0]
    mol_e_t = np.round(mol_dataPoint[1],2)

    dict = {}
    dict['Descriptors'] = ['ID', 'Smiles', 'Triplet energy [kcal/mol]']
    dict['Value'] = [mol_id, mol_SMILES, mol_e_t]
    return pd.DataFrame(dict)

#--------------Prepare the 3D plot-------------------#

data, data_transpose, mol_pics = data_array.transpose(), data_array, pics
plot_df = pd.DataFrame(data)
df = generate_info_dataframe(data_transpose[0][3])

# initiating the app
app = dash.Dash()  # defining the layout

fig = px.scatter_3d(x=data[4],
                    y=data[5],
                    z=data[6],
                    hover_name=data[3],
                    color=data[2],
                    color_discrete_map = {  35: '#e5f5e0',  # light green
                                            40: '#a1d99b',  # soft green
                                            45: '#74c476',  # medium green
                                            50: '#31a354',  # dark green
                                            55: '#006d2c',  # very dark green
                                            60: '#fcbba1',  # light red
                                            65: '#fc9272',  # medium red
                                            70: '#de2d26'},  # dark red
                    labels={'x': 'UMAP_1',
                            'y': 'UMAP_2',
                            'z': 'UMAP_3'},
                    height=900,
                    width=950)

fig.update_traces(marker=dict(size=4,
                              opacity=0.7,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

app.layout = html.Div([

    html.Div([
        dcc.Graph(id='DB-SMILES',
                  figure=fig
                  )],
        style={'width': '60%', 'display': 'inline-block', 'border-style': 'solid', 'vertical-align': 'middle'}),
    html.Div([
        html.Div([
            html.Img(id='mol_pic', src=f'data:image/png;base64,{mol_pics[0]}')
        ], id='mol_pic_div'),
        html.Div([
            dash_table.DataTable(
                id='mol_infotable',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records')
            )], id='mol_infotable_div')], style={'width': '30%', 'display': 'inline-block',
                                                 'margin-left': '50px', 'border-style': 'solid',
                                                 'vertical-align': 'middle'})
])

@app.callback(Output('mol_infotable_div', 'children'), [Input('DB-SMILES', 'hoverData')])
def callback_stats(hoverData):
    if hoverData is None:
        mol_id = 0
    else:
        point = hoverData['points'][0]
        mol_id = int(point['hovertext'])

    out_infos = generate_info_dataframe(mol_id)

    dt = dash_table.DataTable(
        id='mol_infotable',
        columns=[{"name": i, "id": i} for i in out_infos.columns],
        data=out_infos.to_dict('records'),
        style_cell_conditional=[
            {'if': {'column_id': 'Descriptors'},
             'width': '30%'},
            {'if': {'column_id': 'Value'},
             'width': '70%'},
        ],
        style_data={
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 0,
            'whiteSpace': 'normal',
        },
    )

    return dt

@app.callback(Output('mol_pic_div', 'children'), [Input('DB-SMILES', 'hoverData')])
def callback_stats(hoverData):
    if hoverData is None:
        mol_id = int(0)
        return html.Img(
            id='mol_pic',
            src=f'data:image/png;base64,{mol_pics[mol_id]}',
            style={'width': '85%', 'height': 'auto', 'display': 'block', 'margin': 'auto'}
        )
    else:
        point = hoverData['points'][0]
        mol_id = int(point['hovertext'])
        return html.Img(
            id='mol_pic',
            src=f'data:image/png;base64,{mol_pics[mol_id]}',
            style={'width': '85%', 'height': 'auto', 'display': 'block', 'margin': 'auto'}
        )

app.run_server(mode='external')